In [42]:
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from nltk.corpus import stopwords

In [43]:
english_stopwords = stopwords.words("english")
data = pd.read_csv(r"C:\Users\alkrd\Desktop\graduation_project\the_project\preprocssed_data\cleaned_data_USA.csv")
dataFrame = data[['newsSnippet']]

In [44]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    print("lemmatizing...")
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    text_out = []
    for text in texts:
        doc = nlp(text)
        new_text = [token.lemma_ for token in doc if token.pos_ in allowed_postags]
        final = " ".join(new_text)
        text_out.append(final)
    return text_out

lemmatized_texts = lemmatization(dataFrame['newsSnippet'].astype(str).tolist())
print("lemmitized sample: ", lemmatized_texts[100])

lemmatizing...
lemmitized sample:  championship game feature accord most ranking good team result flash brilliance sloppy play defeat night


In [45]:
def gen_words(texts):
    print("removing stop words...")
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)
print("after removing stop words: ",data_words[100])

removing stop words...
after removing stop words:  ['championship', 'game', 'feature', 'accord', 'most', 'ranking', 'good', 'team', 'result', 'flash', 'brilliance', 'sloppy', 'play', 'defeat', 'night']


In [46]:
bigrams_phrases = gensim.models.Phrases(data_words, min_count=3, threshold=80)
trigram_phrases = gensim.models.Phrases(bigrams_phrases[data_words], threshold=80)

bigram = gensim.models.phrases.Phraser(bigrams_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return(bigram[doc] for doc in texts)

def make_tigram(texts):
    return (trigram[bigram[doc]] for doc in texts)

data_bigrams = list(make_bigrams(data_words))
data_bigrams_trigrams = list(make_tigram(data_bigrams))

print(data_bigrams_trigrams)



[['logo', 'stock', 'buy', 'rating', 'restate', 'analyst', 'research', 'note', 'issue', 'currently', 'target', 'price', 'apparel', 'retailer', 'stock'], ['dick', 'expand', 'digital', 'tool', 'acquisition', 'company', 'mobile', 'scorekeeping', 'app', 'same', 'name', 'sporting', 'good', 'company', 'location', 'seem'], ['service', 'directv', 'now', 'include', 'many', 'channel', 'include', 'socalle', 'cordcutter', 'frequently', 'miss', 'ditch', 'cable', 'live', 'television', 'stream', 'mobile', 'device', 'tablet', 'computer'], ['see', 'playoff', 'hope', 'take', 'major', 'blow', 'night', 'many', 'way', 'only', 'blame', 'overcome', 'shoddy', 'defense', 'give', 'average', 'point', 'game'], ['announce', 'release', 'pc', 'develop', 'board', 'game', 'digital', 'version', 'realworld', 'board', 'game', 'create', 'player', 'become', 'train'], ['about', 'dozen', 'hour', 'slain', 'countless', 'monster', 'travel', 'hundred', 'mile', 'bond', 'tightknit', 'group', 'friend', 'strange', 'feeling', 'swell',

In [47]:
from gensim.models import TfidfModel
id2word = corpora.Dictionary(data_bigrams_trigrams)
texts = data_bigrams_trigrams
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[0])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words = []
words_missing_in_tfidf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]  
    corpus[i] = new_bow

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1)]


In [48]:
# print("vectorizing...")
# id2word = corpora.Dictionary(data_words)
# corpus = []
# for text in data_words:
#     new = id2word.doc2bow(text)
#     corpus.append(new)
# print("corpus? ", corpus[0])

In [49]:
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=30,
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=20,
    alpha="auto"
)

In [50]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0     -0.441143  0.143074       1        1  7.402520
15     0.189962 -0.411014       2        1  6.299459
6      0.398570  0.204324       3        1  6.127764
19     0.263294  0.360266       4        1  6.002533
5      0.398301 -0.172529       5        1  5.251282
2     -0.006481 -0.431270       6        1  4.904174
10    -0.176710  0.384833       7        1  4.606354
18     0.391517  0.038747       8        1  4.098309
29    -0.354883 -0.175050       9        1  3.974657
14    -0.383121 -0.033771      10        1  3.927358
24    -0.276825  0.261812      11        1  3.777818
1     -0.127370 -0.344717      12        1  3.585062
12    -0.263693 -0.291104      13        1  3.521357
27     0.298102  0.002017      14        1  3.437733
28     0.041072  0.381964      15        1  3.385015
23     0.237746 -0.247093      16        1  3.091725
3      0.209450 -0.127669      17        1  2.580112
26    -0.251944  0.101720      18        1  2.411897
11    -0.204691 -0.108097      19        1  2.168959
9      0.064214  0.262855      20        1  2.167734
4      0.155557  0.191250      21        1  2.050473
16    -0.069584  0.158785      22        1  2.015211
8     -0.080990  0.259025      23        1  1.980970
13     0.081808 -0.190512      24        1  1.897372
20     0.138749  0.079729      25        1  1.720794
21    -0.103115 -0.173938      26        1  1.717984
22    -0.003618 -0.164789      27        1  1.696527
25    -0.142476  0.014982      28        1  1.496827
7      0.058884 -0.010206      29        1  1.467563
17    -0.040583  0.036378      30        1  1.234456, topic_info=         Term        Freq       Total Category  logprob  loglift
200       say  706.000000  706.000000  Default  30.0000  30.0000
501     first  473.000000  473.000000  Default  29.0000  29.0000
408    season  356.000000  356.000000  Default  28.0000  28.0000
62       take  447.000000  447.000000  Default  27.0000  27.0000
279      just  341.000000  341.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
13     target    0.033984    5.900990  Topic30  -9.7573  -0.7624
15        app    0.033984   13.206145  Topic30  -9.7573  -1.5680
16    company    0.033984   89.297191  Topic30  -9.7573  -3.4793
20       good    0.033984  182.208483  Topic30  -9.7573  -4.1925
21   location    0.033984   35.236709  Topic30  -9.7573  -2.5494

[1122 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
75       10  0.982819     about
606      12  0.991166    accord
411       9  0.951489   account
744      24  0.929033    accuse
168       1  0.967831       act
...     ...       ...       ...
958      24  0.682956     worry
885       7  0.975182     write
149      18  0.918103    writer
228       1  0.996908      year
523       3  0.981771  year_ago

[641 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 16, 7, 20, 6, 3, 11, 19, 30, 15, 25, 2, 13, 28, 29, 24, 4, 27, 12, 10, 5, 17, 9, 14, 21, 22, 23, 26, 8, 18])